# Problem Satement : To Perform SVM, RF, Boosting, KNN On EEG Emotion Dataset

## 1.1 Importing Basic Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
 

# 1.1.1 DatsSet Information

## <a id='var_des'>2.2. Variables description</a>

* `trial number`: number of the trial (obviously) 
* `sensor postition`:  position of electrode placed on subject's scalp (based on International 10-20 system)
<img src="https://i.ibb.co/M2GRQNz/670px-International-10-20-system-for-EEG-MCN-svg.png" alt="670px-International-10-20-system-for-EEG-MCN-svg" border="0">
* `sample num`: 0-255
* `sensor value`: value in microvolts 
* `subject identifier`: `a` - Alcoholic; `c` - Control
* `matching condition`: 
 * `S1 obj` - a single object shown; 
 * `S2 match` - object 2 shown in a matching condition (S1 was identical to S2), 
 * `S2 nomatch` - object 2 shown in a non matching condition (S1 differed from S2)
* `channel`: channel number (0-63). Basically, it's the same as `sensor position` column, so one of these columns can be dropped
* `name`: a serial code assigned to each subject
* `time`: inverse of `sample num` measured in seconds


I have changed some `sensor position`s so they match the basics for head topography visualization. Also, I have removed those positions which equal to **X**, **Y** and **ND** since I couldnt figure out which regions they respond to.

## 1.2 Reading Dataset

In [2]:
df=pd.read_csv("EEG_data.csv")
df.head()

,video_id,subject_id,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,...,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma,subject_understood
0,0,0,4210.641113,4179.102539,4287.948730,4235.384766,4207.948730,4165.000000,4135.897461,4170.000000,...,1.583895,0.504567,0.471979,0.138717,1.801014,1.504794,0.258570,0.435745,0.469483,0
1,0,0,4201.025879,4188.717773,4280.128418,4236.922852,4209.615234,4152.436035,4130.128418,4149.487305,...,1.709560,0.606587,0.527616,0.155580,1.859177,1.379617,0.317579,0.468416,0.642560,0
2,0,0,4203.205078,4182.820313,4282.820313,4231.025879,4207.820313,4172.436035,4131.538574,4147.948730,...,1.873591,0.795834,0.565414,0.170816,2.027946,1.283876,0.441925,0.494701,0.798197,0
3,0,0,4186.538574,4168.717773,4266.794922,4229.230957,4202.179688,4155.384766,4128.333496,4151.666504,...,2.110017,1.021118,0.579656,0.180056,2.265952,1.306188,0.616881,0.506062,0.886495,0
4,0,0,4232.436035,4216.922852,4306.922852,4270.769043,4217.436035,4166.538574,4155.897461,4162.820313,...,2.462552,1.230984,0.573620,0.181081,2.461205,1.522420,0.822598,0.498361,0.874455,0


In [3]:
df.columns

Index(['video_id', 'subject_id', 'EEG.AF3', 'EEG.F7', 'EEG.F3', 'EEG.FC5',
       'EEG.T7', 'EEG.P7', 'EEG.O1', 'EEG.O2', 'EEG.P8', 'EEG.T8', 'EEG.FC6',
       'EEG.F4', 'EEG.F8', 'EEG.AF4', 'POW.AF3.Theta', 'POW.AF3.Alpha',
       'POW.AF3.BetaL', 'POW.AF3.BetaH', 'POW.AF3.Gamma', 'POW.F7.Theta',
       'POW.F7.Alpha', 'POW.F7.BetaL', 'POW.F7.BetaH', 'POW.F7.Gamma',
       'POW.F3.Theta', 'POW.F3.Alpha', 'POW.F3.BetaL', 'POW.F3.BetaH',
       'POW.F3.Gamma', 'POW.FC5.Theta', 'POW.FC5.Alpha', 'POW.FC5.BetaL',
       'POW.FC5.BetaH', 'POW.FC5.Gamma', 'POW.T7.Theta', 'POW.T7.Alpha',
       'POW.T7.BetaL', 'POW.T7.BetaH', 'POW.T7.Gamma', 'POW.P7.Theta',
       'POW.P7.Alpha', 'POW.P7.BetaL', 'POW.P7.BetaH', 'POW.P7.Gamma',
       'POW.O1.Theta', 'POW.O1.Alpha', 'POW.O1.BetaL', 'POW.O1.BetaH',
       'POW.O1.Gamma', 'POW.O2.Theta', 'POW.O2.Alpha', 'POW.O2.BetaL',
       'POW.O2.BetaH', 'POW.O2.Gamma', 'POW.P8.Theta', 'POW.P8.Alpha',
       'POW.P8.BetaL', 'POW.P8.BetaH', 'POW.P8.Gamma', 'P

In [4]:
df.shape

(68831, 87)

**Here Observation=>68831, Feature=> 87**

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68831 entries, 0 to 68830
Data columns (total 87 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   video_id            68831 non-null  int64  
 1   subject_id          68831 non-null  int64  
 2   EEG.AF3             68831 non-null  float64
 3   EEG.F7              68831 non-null  float64
 4   EEG.F3              68831 non-null  float64
 5   EEG.FC5             68831 non-null  float64
 6   EEG.T7              68831 non-null  float64
 7   EEG.P7              68831 non-null  float64
 8   EEG.O1              68831 non-null  float64
 9   EEG.O2              68831 non-null  float64
 10  EEG.P8              68831 non-null  float64
 11  EEG.T8              68831 non-null  float64
 12  EEG.FC6             68831 non-null  float64
 13  EEG.F4              68831 non-null  float64
 14  EEG.F8              68831 non-null  float64
 15  EEG.AF4             68831 non-null  float64
 16  POW.

* **All Feature is numeric**

In [6]:
df.describe()

,video_id,subject_id,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,...,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma,subject_understood
count,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,...,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000,68831.000000
mean,5.624297,3.710828,4276.496212,4257.994590,4302.803482,4292.027479,4302.093465,4243.397476,4231.846947,4240.153361,...,5.386578,2.847343,1.356892,0.770303,22.591861,8.133554,3.861532,1.520038,0.697575,0.789906
std,2.215993,1.828410,115.584631,140.884855,113.720961,135.578849,117.727171,118.189886,115.590762,137.366806,...,42.886793,36.480798,17.930204,8.991064,150.716768,51.722390,35.243356,13.153730,3.937423,0.407378
min,0.000000,0.000000,1030.769287,805.384644,1320.769287,806.538452,1904.487183,1710.769287,1794.871826,1466.538452,...,0.051299,0.036999,0.062924,0.043833,0.195020,0.077566,0.041230,0.056951,0.035907,0.000000
25%,5.000000,3.000000,4258.461426,4236.410156,4283.205078,4273.077148,4282.436035,4220.769043,4211.602539,4218.205078,...,1.093447,0.619023,0.340121,0.166746,1.840644,0.997642,0.511406,0.284404,0.139540,1.000000
50%,6.000000,4.000000,4287.307617,4266.538574,4305.897461,4297.436035,4312.179688,4253.846191,4244.487305,4252.948730,...,1.921246,1.032671,0.612935,0.366483,3.317688,1.679386,0.833078,0.472978,0.262676,1.000000
75%,6.000000,4.000000,4311.153809,4296.153809,4327.692383,4322.563965,4342.179688,4282.820313,4271.794922,4279.615234,...,3.587066,1.850553,1.076909,0.708885,7.129672,3.035433,1.464920,0.837565,0.516598,1.000000
max,10.000000,7.000000,6238.077148,7599.615234,6291.794922,7600.128418,7599.615234,6695.641113,7525.128418,7611.025879,...,3825.365967,3803.180664,2010.126343,966.228027,8159.436035,3092.790283,3372.114746,1347.074341,331.578979,1.000000


## 2. Handling  Null Value

In [7]:
df.isnull().sum()

video_id              0
subject_id            0
EEG.AF3               0
EEG.F7                0
EEG.F3                0
                     ..
POW.AF4.Alpha         0
POW.AF4.BetaL         0
POW.AF4.BetaH         0
POW.AF4.Gamma         0
subject_understood    0
Length: 87, dtype: int64

* **No Null value is present**

# 3. SVM

In [3]:
x=df.iloc[:,0:-1]
y=df.iloc[:,-1]

In [9]:
x

,video_id,subject_id,EEG.AF3,EEG.F7,EEG.F3,EEG.FC5,EEG.T7,EEG.P7,EEG.O1,EEG.O2,...,POW.F8.Theta,POW.F8.Alpha,POW.F8.BetaL,POW.F8.BetaH,POW.F8.Gamma,POW.AF4.Theta,POW.AF4.Alpha,POW.AF4.BetaL,POW.AF4.BetaH,POW.AF4.Gamma
0,0,0,4210.641113,4179.102539,4287.948730,4235.384766,4207.948730,4165.000000,4135.897461,4170.000000,...,3.758370,1.583895,0.504567,0.471979,0.138717,1.801014,1.504794,0.258570,0.435745,0.469483
1,0,0,4201.025879,4188.717773,4280.128418,4236.922852,4209.615234,4152.436035,4130.128418,4149.487305,...,4.580270,1.709560,0.606587,0.527616,0.155580,1.859177,1.379617,0.317579,0.468416,0.642560
2,0,0,4203.205078,4182.820313,4282.820313,4231.025879,4207.820313,4172.436035,4131.538574,4147.948730,...,5.230789,1.873591,0.795834,0.565414,0.170816,2.027946,1.283876,0.441925,0.494701,0.798197
3,0,0,4186.538574,4168.717773,4266.794922,4229.230957,4202.179688,4155.384766,4128.333496,4151.666504,...,5.472836,2.110017,1.021118,0.579656,0.180056,2.265952,1.306188,0.616881,0.506062,0.886495
4,0,0,4232.436035,4216.922852,4306.922852,4270.769043,4217.436035,4166.538574,4155.897461,4162.820313,...,5.209275,2.462552,1.230984,0.573620,0.181081,2.461205,1.522420,0.822598,0.498361,0.874455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68826,10,7,4353.846191,4349.487305,4317.563965,4343.461426,4375.641113,4296.025879,4351.666504,4309.358887,...,1.829682,2.088463,0.925704,0.825126,0.576437,2.009375,3.020382,2.115835,1.735570,1.125858
68827,10,7,4335.769043,4326.538574,4310.641113,4331.153809,4363.333496,4297.179688,4336.025879,4282.563965,...,2.820868,1.899325,0.833753,0.786224,0.600227,2.505607,2.680849,1.971062,1.468342,1.250918
68828,10,7,4321.666504,4313.205078,4310.000000,4310.384766,4357.436035,4296.025879,4337.563965,4291.153809,...,4.052598,1.743110,0.789346,0.777936,0.631701,3.082165,2.443027,1.868980,1.213362,1.373250
68829,10,7,4334.615234,4316.666504,4318.717773,4339.102539,4370.512695,4297.436035,4333.461426,4297.051270,...,5.277864,1.603054,0.781398,0.792152,0.661582,3.607407,2.341642,1.780673,0.982586,1.449505


In [10]:
y

0        0
1        0
2        0
3        0
4        0
        ..
68826    1
68827    1
68828    1
68829    1
68830    1
Name: subject_understood, Length: 68831, dtype: int64

In [4]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)

### 3.2 Feature Scaling

In [12]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

### 3.3 Model Train

In [13]:
from sklearn.svm import SVR
regression=SVR(kernel='rbf')
regression.fit(x_train,y_train)

SVR()

In [14]:
y_pred=regression.predict(x_test)
y_pred

array([0.97746707, 0.74914846, 0.12613753, ..., 1.00443318, 0.96565135,
       0.76175015])

In [15]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_pred))


0.9321556465024832


#### Accuracy is 93.215%

## 3.4 Hyperparameter Tuning With SVR

In [16]:
from sklearn.model_selection import GridSearchCV
 
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel':['linear'],
              'epsilon':[0.1,0.2,0.3]
              }

In [17]:
grid=GridSearchCV(SVR(),param_grid=param_grid,refit=True,cv=5,verbose=3)

In [ ]:
grid.fit(x_train,y_train)

In [ ]:
grid.best_params_

In [ ]:
y_pred_2=grid.predict(x_test)
print(r2_score(y_test,y_pred))

# 2. Random Forest Algo

In [5]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 300, max_features = 'sqrt', max_depth = 5, random_state = 18)
rf.fit(x_train, y_train)

RandomForestRegressor(max_depth=5, max_features='sqrt', n_estimators=300,
                      random_state=18)

In [8]:
y_pred2=rf.predict(x_test)

In [9]:
from sklearn.metrics import r2_score
print(r2_score(y_test,y_pred2))

0.8230901300381779


* **So By Using Random Forest we get 82.30% Accuracy**

# 3. KNN

In [10]:
from sklearn.neighbors import KNeighborsRegressor

In [11]:
regression_knn=KNeighborsRegressor(n_neighbors=6,algorithm='auto')
regression_knn.fit(x_train,y_train)

KNeighborsRegressor(n_neighbors=6)

In [12]:
y_pred3=regression_knn.predict(x_test)

In [13]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [14]:
print(r2_score(y_test,y_pred3))


0.7871555030897534


* **By using KNN we get 78.71% Accuracy**

In [15]:
print(mean_absolute_error(y_test,y_pred3))
print(mean_squared_error(y_test,y_pred3))

0.06765133171912834
0.0350739843960183


# 4. XGBOOST

In [17]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xb

### 4.1 AdaBoost

In [18]:
regression_ada=AdaBoostRegressor()
regression_ada.fit(x_train,y_train)

AdaBoostRegressor()

In [19]:
y_pred4=regression_ada.predict(x_test)

In [24]:
from sklearn.metrics import r2_score

In [25]:
print(r2_score(y_test,y_pred4))


0.95586550306895


* **By Using AdaBoostRegressor We get 95.58% Accuracy**

## 4.2 XGBoost

In [27]:
regressor_xg=xb.XGBRegressor()
regressor_xg.fit(x_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [28]:
y_pred5=regressor_xg.predict(x_test)

In [29]:
print(r2_score(y_test,y_pred5))


0.9990830378663655


* **By Using XgBoost We get 99.90% Accuracy**

# Conclusion:
**After Analysis of EEG dataset using SVM,RF,KNN,Boosting
XgBoost Have Highest Accuracy 99.90%**